***
<span style="font-size:32px; color:rgba(0, 0, 255, 0.5);">Day 3 - Generative AI Agents</span>

---

<table style="width: 100%;">
  <tr>
    <td style="background-color: rgba(0, 255, 0, 0.2); text-align: center; font-size: 16px;">
    </td>
  </tr>
</table>

<span style="font-size:24px; color:rgba(0, 0, 0, 0.5);">Function calling with the Gemini API</span>

---
Generative AI models can be trained to use tools to access real-time information or suggest a real-world action. For example, a model can leverage a  database retrieval tool to access specific information, like a customer's purchase history, so it can generate tailored shopping recommendations. lternatively, based on a user's query, a model can make various API calls to send an email response to a colleague or complete a financial transaction on your behalf. To do so, the model must not only have access to a set of external tools, it needs the ability to plan and execute any task in a self-
directed fashion. This combination of reasoning, logic, and access to external information that are all connected to a Generative AI model invokes the concept of an agent, or a program that extends beyond the standalone capabilities of a Generative AI model. This whitepaper dives into all these and associated aspects in more detail.

**Authors**<br>
Julia Wiesinger, Patrick Marlow and Vladimir Vuskovic

<span style="font-size:18px; color:rgba(0, 0, 0, 0.5);">Resources</span>

---
**Whitepaper**<br>
https://www.kaggle.com/whitepaper-agents

**Embedding and Vector Stores Podcast**<br>
https://www.youtube.com/watch?v=H4gZd4BCrDQ

**Embedding and Vector Databases Livestream**<br>
https://www.youtube.com/watch?v=HQUtMWoTAD4

**Get your API key from**<br>
https://aistudio.google.com/app/apikey

**Kaggle**<br>
https://www.kaggle.com/code/markishere/day-3-function-calling-with-the-gemini-api<br>

In [1]:
# %pip install google-generativeai ipython-sql

<span style="font-size:18px; color:rgba(0, 0, 0, 0.5);">Libraries</span>

---

In [12]:
import os, sqlite3, textwrap

from dotenv import load_dotenv

import google.generativeai as genai

<span style="font-size:18px; color:rgba(0, 0, 0, 0.5);">Initialize the API</span>

---

In [13]:
# Load API key from .env file
load_dotenv()
api_key = os.getenv("GAI_API_KEY")

# Set up the API key for the genai library
genai.configure(api_key=api_key)

<span style="font-size:18px; color:rgba(0, 0, 0, 0.5);">Create a local database</span>

---

In [14]:
%load_ext sql
%sql sqlite:///sample.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


<span style="font-size:18px; color:rgba(0, 0, 0, 0.5);">Create tables and insert synthetic data</span>

---

In [15]:
%%sql
-- Create the 'products' table
CREATE TABLE IF NOT EXISTS products (
  	product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	product_name VARCHAR(255) NOT NULL,
  	price DECIMAL(10, 2) NOT NULL
  );

-- Create the 'staff' table
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL
  );

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );

-- Insert data into the 'products' table
INSERT INTO products (product_name, price) VALUES
  	('Laptop', 799.99),
  	('Keyboard', 129.99),
  	('Mouse', 29.99);

-- Insert data into the 'staff' table
INSERT INTO staff (first_name, last_name) VALUES
  	('Alice', 'Smith'),
  	('Bob', 'Johnson'),
  	('Charlie', 'Williams');

-- Insert data into the 'orders' table
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
  	('David Lee', 1, 1),
  	('Emily Chen', 2, 2),
  	('Frank Brown', 1, 3);

 * sqlite:///sample.db
Done.
Done.
Done.
3 rows affected.
3 rows affected.
3 rows affected.


[]

<span style="font-size:18px; color:rgba(0, 0, 0, 0.5);">Define database functions</span>

---

Function calling with Gemini API's Python SDK can be implemented by defining an OpenAPI schema that is passed to the model. Alternatively you can define Python functions and let the SDK inspect them to automatically define the schema. In this latter case, it's important that the functions are type annotated and have accurate docstrings that describe what the functions do - the model has no insight into the function body, so the docs function as the interface.

By providing three key pieces of functionality - listing tables, describing a table, and executing a query - the LLM (or even another user) will have the basic tools needed to understand and interrogate the database.

Start with a database connection that will be used across all of the functions.

In [17]:
db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

The first function will list all tables available in the database. Define it, and test it out to ensure it works.

In [18]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables


['products', 'sqlite_sequence', 'staff', 'orders']

Once the available tables is known, the next step a database user will need is to understand what columns are available in a given table. Define that too, and test that it works as expected.

In [19]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(' - DB CALL: describe_table')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("products")

 - DB CALL: describe_table


[('product_id', 'INTEGER'),
 ('product_name', 'VARCHAR(255)'),
 ('price', 'DECIMAL(10, 2)')]

Now that the system knows what tables and columns are present, it has enough information to be able to generate and run a SELECT query. Now provide that functionality, and test that it works.

In [20]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute a SELECT statement, returning the results."""
    print(' - DB CALL: execute_query')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("select * from products")

 - DB CALL: execute_query


[(1, 'Laptop', 799.99),
 (2, 'Keyboard', 129.99),
 (3, 'Mouse', 29.99),
 (4, 'Laptop', 799.99),
 (5, 'Keyboard', 129.99),
 (6, 'Mouse', 29.99),
 (7, 'Laptop', 799.99),
 (8, 'Keyboard', 129.99),
 (9, 'Mouse', 29.99)]

<span style="font-size:18px; color:rgba(0, 0, 0, 0.5);">Implement function calls</span>

---
Function calling works by adding specific messages to a chat session. When function schemas are defined and made available to the model and a conversation is started, instead of returning a text response, the model may return a function_call instead. When this happens, the client must respond with a function_response, indicating the result of the call, and the conversation can continue on as normal.

This function calling interaction normally happens manually, allowing you, the client, to validate and initiate the call. However the Python SDK also supports automatic function calling, where the supplied functions will be automatically invoked. This is a powerful feature and should only be exposed when it is safe to do so, such as when the functions have no side-effects).

Here's the state diagram representing the conversation flow with function calling. With automatic function calling, the bottom row is executed automatically by the Python SDK. In manual function calling, you write the code to run each step individually.

![Alt Text](./images/function_call_diagram.png)

In [21]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database for a computer
store. You will take the users questions and turn them into SQL queries using the tools
available. Once you have the information you need, you will answer the user's question using
the data returned. Use list_tables to see what tables are present, describe_table to understand
the schema, and execute_query to issue an SQL SELECT query."""

model = genai.GenerativeModel(
    "models/gemini-1.5-flash-latest", tools=db_tools, system_instruction=instruction
)

# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# Start a chat with automatic function calling enabled.
chat = model.start_chat(enable_automatic_function_calling=True)

Now you can engage in a chat conversation where you can ask about the contents of the database.

In [22]:
resp = chat.send_message("What is the cheapest product?", request_options=retry_policy)
print(resp.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: execute_query
The cheapest product is the Mouse, costing $29.99.



If you re-use the same ChatSession object, the conversation will continue statefully. If you wish to start fresh, you can call start_chat again, or call rewind on the chat object to go back a turn.

Continue the chat here by asking a follow-up question. Note that the database information is preserved, and the context of the specific product is inferred.

In [23]:
resp = chat.send_message("and how much is it?", request_options=retry_policy)
print(resp.text)

The Mouse costs $29.99.



Explore the chat session and ask your own questions. If you want to try asking more complex questions, try using the gemini-1.5-pro model. It has a lower rate limit, so calls might take a bit longer on the free tier, but you'll notice an improvement in expressivity.

In [24]:
model = genai.GenerativeModel(
    "models/gemini-1.5-pro-latest", tools=db_tools, system_instruction=instruction
)

chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('Which salesperson sold the cheapest product?', request_options=retry_policy)
print(response.text)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: describe_table
 - DB CALL: execute_query
Alice Smith sold the cheapest product.


<span style="font-size:18px; color:rgba(0, 0, 0, 0.5);">Inspecting the conversation</span>

---
To see the calls that the model makes, and what the client returns in response, you can inspect chat.history. This helper function will print out each turn along with the relevant fields passed or returned.

In [25]:
def print_chat_turns(chat):
    """Prints out each turn in the chat history, including function calls and responses."""
    for event in chat.history:
        print(f"{event.role.capitalize()}:")

        for part in event.parts:
            if txt := part.text:
                print(f'  "{txt}"')
            elif fn := part.function_call:
                args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
                print(f"  Function call: {fn.name}({args})")
            elif resp := part.function_response:
                print("  Function response:")
                print(textwrap.indent(str(resp), "    "))

        print()


print_chat_turns(chat)

User:
  "Which salesperson sold the cheapest product?"

Model:
  Function call: list_tables()

User:
  Function response:
    name: "list_tables"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              string_value: "products"
            }
            values {
              string_value: "sqlite_sequence"
            }
            values {
              string_value: "staff"
            }
            values {
              string_value: "orders"
            }
          }
        }
      }
    }


Model:
  Function call: describe_table(table_name=products)

User:
  Function response:
    name: "describe_table"
    response {
      fields {
        key: "result"
        value {
          list_value {
            values {
              list_value {
                values {
                  string_value: "product_id"
                }
                values {
                  string_value: "INTEGER"
                }


In this output you can see each of the conversational turns that were made. Note that the model doesn't remember anything outside of a ChatSession object, so you can make changes to the database structure or data and the model will respond without needing any code changes - try this out!

<span style="font-size:18px; color:rgba(0, 0, 0, 0.5);">Further reading</span>

---
To learn more about what the Gemini API can do with function calling, check out the Function calling cookbook (see Manual function calling to understand how function calling works manually) as well as Function calling config, which gives you fine-grained control over how function calling is triggered.